# Get Ancestral Alleles, Allele Frequencies in Modern Humans, and Variants in Introgressed Haplotypes

We have a dataframe on potential splice variants in archaics. Celebrate good times. Now let's add some more information. We need data on the ancestral allele and allele frequencies in humans for all variants including non-SA sites, InDels, and the X chromosome. We will also concatenate introgressed tag SNPs from Vernot et al. 2016. Finally, we will liftOver SA autosomal SNV variants to see if they also occur in GTex as sQTLs.

Change directories. 

In [1]:
cd ../../data/archaic_variants

: 1

## Ancestral Alleles

In [2]:
awk '{print $1,$2-1,$2}' OFS='\t' ../dataframes/archaic_data_with_constraint.txt  > all_variant_sites_hg19.bed

: 1

LiftOver to hg38, save as 'all_variant_sites_hg38.bed', remove 'chr', and run the script below.

In [3]:
sed -e 's/chr//g' all_variant_sites_hg38.bed > all_variant_sites_hg38_no_chr.bed

: 1

A couple of variants mapped to small contigs. Let's remove those.

In [4]:
awk 'length($1)<=2' OFS='\t' all_variant_sites_hg38_no_chr.bed > variant_sites_hg38_no_chr.bed

: 1

In [5]:
#qsub ../4_modern_data_preparation/call_ancestral_allele.sh

: 1

In [6]:
awk '{print "chr"$1,$2,$3}' OFS='\t' variant_sites_hg38_with_ancestral.bed  > variant_sites_hg38_with_ancestral_chrom_pos.bed
awk '{print $4}' OFS='\t' variant_sites_hg38_with_ancestral.bed  > variant_sites_hg38_with_ancestral_allele_only.bed

: 1

Run variant_sites_hg38_with_ancestral_chrom_pos.bed through LiftOver and convert back to hg19, saving the file as 'variant_sites_hg19_with_ancestral_chrom_pos.bed'. Several sites failed. Why? Not a clue! Nonetheless, we will need to remove their alleles from the allele file based on row number. Let's put those LiftOver failures in a file: 'hg38_to_hg19_failed.txt'. Get the rows numbers first.

In [7]:
grep -n -f hg38_to_hg19_failed.txt variant_sites_hg38_with_ancestral.bed

99930:1	149074811	149074812	.
100245:1	148920129	148920130	-
100615:1	145812670	145812671	c
101366:1	150131675	150131676	T
221828:10	45974162	45974163	T
463249:12	80454578	80454579	G
669248:15	64954742	64954743	-
807413:17	37607973	37607974	A
807476:17	37694437	37694438	-
837140:17	74870601	74870602	T
942063:19	39906447	39906448	C
942064:19	39906464	39906465	G
942065:19	39906484	39906485	C
942066:19	39906513	39906514	G
942067:19	39906676	39906677	G
942068:19	39906676	39906677	G
942069:19	39906703	39906704	G
942070:19	39907217	39907218	C
942071:19	39907545	39907546	T
942072:19	39907663	39907664	G
946093:19	44489835	44489836	A
957603:19	54360777	54360778	A
1767426:7	72735865	72735866	T
1875435:8	22109194	22109195	c
1992078:9	61101954	61101955	.
1992086:9	41160429	41160430	-
2076681:X	49190519	49190520	-
2089596:X	105219597	105219598	T
2089597:X	105219597	105219598	T


: 1

Now delete those alleles by row number.

In [8]:
sed '99930d;100245d;100615d;101366d;221828d;463249d;669248d;807413d;807476d;837140d;942063d;942064d;942065d;942066d;942067d;942068d;942069d;942070d;942071d;942072d;946093d;957603d;1767426d;1875435d;1992078d;1992086d;2076681d;2089596d;2089597d' variant_sites_hg38_with_ancestral_allele_only.bed > variant_sites_hg38_with_ancestral_allele_nofailed.bed

: 1

Now we can paste everything together.

In [9]:
paste variant_sites_hg19_with_ancestral_chrom_pos.bed variant_sites_hg38_with_ancestral_allele_nofailed.bed > variant_sites_hg19_with_ancestral.bed

: 1

## Allele Frequencies

In [1]:
#cd ../archaic_variants_in_humans # use if continuing script
cd ../../data/archaic_variants_in_humans # use if starting here

: 1

We've already mapped our variants onto hg38 so now we need the allele frequencies from variants that fall within gene bodies in 1000 genomes. Run the two scripts below. The first will normalize, concatenate, and subset 1000 Genomes variants to those that fall within gene bodies. The second further subsets those data to the archaic variants and then queries for allele frequencies. Some of the alleles from the bcftools output may not actually occur in the archaic data because bcftools only uses chrom and pos, rather than those combined with the reference and alternate allele. However, we're going to later merge the data using all four fields so we can worry about that in the next notebook.

In [2]:
#qsub ../../scripts/4_modern_data_preparation/subset_1KG.sh

: 1

In [3]:
#qsub ../../scripts/4_modern_data_preparation/query_1KG_allele_frequencies.sh

: 1

Now let's split things up for lifting back over to hg19.

In [4]:
awk '{print "chr"$1,$2-1,$2}' OFS='\t' allele_frequencies_hg38.txt  > allele_frequencies_hg38_chrom_pos.bed
awk '{print $3,$4,$5,$6,$7,$8,$9,$10,$11,$12}' OFS='\t' allele_frequencies_hg38.txt > allele_frequencies_hg38_no_chrom_pos.txt

: 1

Run LiftOver and put everything back together. A couple of sites failed again so let's remove those alleles and frequencies based on row number.

chr1	150131675	150131676
chr1	150131675	150131676
chr10	115187172	115187173
chr17	74870601	74870602

In [5]:
grep -n $'1\t150131676' allele_frequencies_hg38.txt
grep -n $'1\t150131676' allele_frequencies_hg38.txt
grep -n $'10\t115187173' allele_frequencies_hg38.txt
grep -n $'17\t74870602' allele_frequencies_hg38.txt

58173:1	150131676	T	A	56	5096	0.01	0.01	0.01	0.02	0.01	0
58174:1	150131676	T	TAAAA	5093	5096	1	1	1	1	1	1
58173:1	150131676	T	A	56	5096	0.01	0.01	0.01	0.02	0.01	0
58174:1	150131676	T	TAAAA	5093	5096	1	1	1	1	1	1
169827:10	115187173	AAT	A	228	5096	0.04	0	0	0.16	0.02	0
503770:17	74870602	T	C	4193	5096	0.82	0.63	0.8	0.98	0.85	0.82


: 1

In [6]:
sed '58173d;58174d;169827d;503770d' allele_frequencies_hg38_no_chrom_pos.txt > allele_frequencies_hg19_no_chrom_pos_no_failed.txt

: 1

In [7]:
paste allele_frequencies_hg19_chrom_pos.bed allele_frequencies_hg19_no_chrom_pos_no_failed.txt > allele_frequencies_hg19.bed

: 1

## Non-ASW AFR Allele Frequencies

Now let's get the allele frequencies for the 1KG AFR superpopulation without ASW. We need samples from ESN, GWD, LWK, MSL, and YRI. The thousand genomes directory has a text file with all non-ASW sample names. We will subset the VCF using bcftools and query for the allele count and allele number using the script below.

In [1]:
cd ../../data/archaic_variants_in_humans # use if starting here

: 1

In [2]:
#qsub ../../scripts/4_modern_data_preparation/calculate_non_ASW_AFR_allele_frequencies.sh

: 1

Now let's LiftOver using the same procedure as before.

In [3]:
awk '{print "chr"$1,$2-1,$2}' OFS='\t' non_ASW_AFR_allele_frequencies_hg38.txt  > non_ASW_AFR_allele_frequencies_hg38_chrom_pos.bed
awk '{print $3,$4,$5,$6}' OFS='\t' non_ASW_AFR_allele_frequencies_hg38.txt > non_ASW_AFR_allele_frequencies_hg38_no_chrom_pos.txt

: 1

The same sites should have failed. Let's remove these before stitching everything together.

In [4]:
grep -n $'1\t150131676' non_ASW_AFR_allele_frequencies_hg38.txt
grep -n $'1\t150131676' non_ASW_AFR_allele_frequencies_hg38.txt
grep -n $'10\t115187173' non_ASW_AFR_allele_frequencies_hg38.txt
grep -n $'17\t74870602' non_ASW_AFR_allele_frequencies_hg38.txt

58173:1	150131676	T	A	19	1026
58174:1	150131676	T	TAAAA	1025	1026
58173:1	150131676	T	A	19	1026
58174:1	150131676	T	TAAAA	1025	1026
169827:10	115187173	AAT	A	168	1026
503770:17	74870602	T	C	1005	1026


: 1

In [5]:
sed '58173d;58174d;169827d;503770d' non_ASW_AFR_allele_frequencies_hg38_no_chrom_pos.txt > non_ASW_AFR_allele_frequencies_hg19_no_chrom_pos_no_failed.txt

: 1

In [6]:
paste non_ASW_AFR_allele_frequencies_hg19_chrom_pos.bed non_ASW_AFR_allele_frequencies_hg19_no_chrom_pos_no_failed.txt > non_ASW_AFR_allele_frequencies_hg19.bed

: 1

## Introgression

Now let's gather introgressed tag SNPs from Vernot et al. 2016. We will worry about allele matching in the next notebook when we merge the data.

In [8]:
#qsub ../../../scripts/4_modern_data_preparation/concat_introgressed_variants.sh

: 1

## sQTLs

Now let's get sQTLs from GTEx.

In [1]:
#cd ../gtex_sqtls # use if continuing script
cd ../../data/gtex_sqtls # use if starting here

: 1

First, create an index file.

In [6]:
awk '{print $1"_"$3}' OFS='\t' ../archaic_variants/all_variant_sites_hg38.bed > hg38_chrom_pos_index.txt

: 1

Generate a long form datafile with chrom and pos as an index for each sQTL tissue from GTEx significant pair files.

In [ ]:
#qsub ../../scripts/4_modern_data_preparation/concat_sQTLs.sh